In [1]:
import pandas as pd
import numpy as np

def calculate_eps_change(df_current, df_previous):
    """
    Tính EPS change giữa năm hiện tại và năm trước
    
    Parameters:
    -----------
    df_current : pandas.DataFrame
        DataFrame năm hiện tại
    df_previous : pandas.DataFrame  
        DataFrame năm trước
    
    Returns:
    --------
    pandas.DataFrame
        DataFrame với cột EPS_change đã được thêm
    """
    df_result = df_current.copy()
    
    # Merge để lấy EPS năm trước
    eps_prev = df_previous[['Mã', 'eps']].rename(columns={'eps': 'eps_prev'})
    df_result = df_result.merge(eps_prev, on='Mã', how='left')
    
    # Tính EPS change: (EPS_t - EPS_t-1) / EPS_t-1
    df_result['EPS_change'] = (df_result['eps'] - df_result['eps_prev']) / df_result['eps_prev']
    
    # Xóa cột tạm
    df_result = df_result.drop('eps_prev', axis=1)
    
    return df_result


def screen_stocks(df_current, df_previous=None, year=None):
    
    df = df_current.copy()
    
    print(f"=== LỌC CỔ PHIẾU NĂM {year if year else ''} ===")
    print(f"Tổng số cổ phiếu ban đầu: {len(df)}")
    
    # Bước 1: Lọc marketcap > 1,000,000
    df = df[df['marketcap'] > 1000000]
    print(f"Sau lọc marketcap > 1,000,000: {len(df)} cổ phiếu")
    
    # Bước 2: Tính EPS change và lọc > 0 (nếu có dữ liệu năm trước)
    if df_previous is not None:
        df = calculate_eps_change(df, df_previous)
        # Lọc EPS change > 0 và không phải NaN/inf
        df = df[(df['EPS_change'] > 0) & 
                (df['EPS_change'].notna()) & 
                (~np.isinf(df['EPS_change']))]
        print(f"Sau lọc EPS_change > 0: {len(df)} cổ phiếu")
    else:
        df = df[df['eps'] > 2000]
        print("Không có dữ liệu năm trước -> Bỏ qua điều kiện EPS_change, sử dụng eps > 2000")
    
    #Bước 3: Tính PE trung bình theo ngành và lọc
    if 'Ngành ICB - cấp 2' in df.columns and 'pe' in df.columns:
        # Tính PE trung bình theo ngành (bỏ qua NaN và inf)
        sector_pe_mean = df.groupby('Ngành ICB - cấp 2')['pe'].apply(
            lambda x: x.replace([np.inf, -np.inf], np.nan).mean()
        ).reset_index()
        sector_pe_mean.columns = ['Ngành ICB - cấp 2', 'sector_pe_avg']
        
        # Merge PE trung bình ngành vào df
        df = df.merge(sector_pe_mean, on='Ngành ICB - cấp 2', how='left')
        
        # Lọc pe < pe trung bình ngành
        df = df[(df['pe'] < df['sector_pe_avg']) & 
                (df['pe'] > 0) & 
                (df['pe'].notna()) & 
                (~np.isinf(df['pe']))]
        print(f"Sau lọc PE < PE trung bình ngành: {len(df)} cổ phiếu")
    else:
        print("Không có cột 'Ngành ICB - cấp 2' hoặc 'pe' -> Bỏ qua điều kiện PE")
    
    # Bước 4: Lọc 1 < PB < 2
    if 'pb' in df.columns:
        df = df[(df['pb'] < 2) & 
                (df['pb'] > 1) & 
                (df['pb'].notna()) & 
                (~np.isinf(df['pb']))]
        print(f"Sau lọc PB < 1: {len(df)} cổ phiếu")
    else:
        print("Không có cột 'pb' -> Bỏ qua điều kiện PB")
    
    # Bước 5: Lọc ROE > 0.15
    if 'roe' in df.columns:
        df = df[(df['roe'] > 0.15) & 
                (df['roe'].notna()) & 
                (~np.isinf(df['roe']))]
        print(f"Sau lọc ROE > 0.17: {len(df)} cổ phiếu")
    else:
        print("Không có cột 'roe' -> Bỏ qua điều kiện ROE")
    
    # Bước 6: Lọc avg20_volume > 100
    if 'avg20_volume' in df.columns:
        df = df[(df['avg20_volume'] > 100) & 
                (df['avg20_volume'].notna()) & 
                (~np.isinf(df['avg20_volume']))]
        print(f"Sau lọc avg20_volume > 100: {len(df)} cổ phiếu")
    else:
        print("Không có cột 'avg20_volume' -> Bỏ qua điều kiện volume")
    
    print(f"Kết quả cuối cùng: {len(df)} cổ phiếu đạt tiêu chí")
    print("-" * 50)
    
    # Sắp xếp theo marketcap giảm dần
    if len(df) > 0:
        df = df.sort_values('marketcap', ascending=False)
        
        # Chọn các cột quan trọng để hiển thị
        display_cols = ['Mã', 'Sàn', 'Ngành ICB - cấp 2', 'marketcap', 'price', 
                       'eps', 'pe', 'pb', 'roe', 'avg20_volume']
        
        # Thêm EPS_change nếu có
        if 'EPS_change' in df.columns:
            display_cols.insert(6, 'EPS_change')
        
        # Thêm sector_pe_avg nếu có
        if 'sector_pe_avg' in df.columns:
            display_cols.insert(-3, 'sector_pe_avg')
        
        # Chỉ lấy các cột tồn tại
        available_cols = [col for col in display_cols if col in df.columns]
        df_display = df[available_cols].copy()
        
        return df_display
    else:
        return pd.DataFrame()


def screen_multiple_years(dataframes_dict):
    """
    Lọc cổ phiếu cho nhiều năm
    
    Parameters:
    -----------
    dataframes_dict : dict
        Dictionary với key là năm, value là DataFrame
        Ví dụ: {2024: df24, 2023: df23, 2022: df22, 2021: df21, 2020: df20}
    
    Returns:
    --------
    dict
        Dictionary chứa kết quả lọc cho từng năm
    """
    results = {}
    sorted_years = sorted(dataframes_dict.keys())
    
    for i, year in enumerate(sorted_years):
        if year in [2020, 2021, 2022, 2023, 2024]:  # Chỉ lọc các năm yêu cầu
            df_current = dataframes_dict[year]
            
            # Tìm năm trước đó có dữ liệu
            df_previous = None
            if i > 0:
                prev_year = sorted_years[i-1]
                df_previous = dataframes_dict[prev_year]
            
            # Lọc cổ phiếu
            result = screen_stocks(df_current, df_previous, year)
            results[year] = result
    
    return results

In [2]:
# Ví dụ sử dụng:
if __name__ == "__main__":
    # Đọc dữ liệu
    df24 = pd.read_csv("2024.csv")
    df23 = pd.read_csv("2023.csv")
    df22 = pd.read_csv("2022.csv")
    df21 = pd.read_csv("2021.csv")
    df20 = pd.read_csv("2020.csv")
    
    # Tạo dictionary
    dataframes = {
        2020: df20,
        2021: df21,
        2022: df22,
        2023: df23,
        2024: df24
    }
    
    # Lọc cổ phiếu cho các năm 2021, 2023, 2024
    screening_results = screen_multiple_years(dataframes)
    
    # Hiển thị kết quả
    for year, result in screening_results.items():
        print(f"\n=== KẾT QUẢ LỌC CỔ PHIẾU NĂM {year} ===")
        if len(result) > 0:
            print(result.round(4))
            print(f"Danh sách mã: {result['Mã'].tolist()}")
        else:
            print("Không có cổ phiếu nào đạt tiêu chí")
        print("\n" + "="*70)
    
    # Lưu kết quả ra file
    for year, result in screening_results.items():
        if len(result) > 0:
            result.to_csv(f"screened_stocks_{year}.csv", index=False, encoding='utf-8-sig')
            print(f"Đã lưu kết quả năm {year} vào file screened_stocks_{year}.csv")

=== LỌC CỔ PHIẾU NĂM 2020 ===
Tổng số cổ phiếu ban đầu: 1664
Sau lọc marketcap > 1,000,000: 340 cổ phiếu
Không có dữ liệu năm trước -> Bỏ qua điều kiện EPS_change, sử dụng eps > 2000
Sau lọc PE < PE trung bình ngành: 80 cổ phiếu
Sau lọc PB < 1: 45 cổ phiếu
Sau lọc ROE > 0.17: 34 cổ phiếu
Sau lọc avg20_volume > 100: 21 cổ phiếu
Kết quả cuối cùng: 21 cổ phiếu đạt tiêu chí
--------------------------------------------------
=== LỌC CỔ PHIẾU NĂM 2021 ===
Tổng số cổ phiếu ban đầu: 1664
Sau lọc marketcap > 1,000,000: 480 cổ phiếu
Sau lọc EPS_change > 0: 291 cổ phiếu
Sau lọc PE < PE trung bình ngành: 216 cổ phiếu
Sau lọc PB < 1: 89 cổ phiếu
Sau lọc ROE > 0.17: 37 cổ phiếu
Sau lọc avg20_volume > 100: 22 cổ phiếu
Kết quả cuối cùng: 22 cổ phiếu đạt tiêu chí
--------------------------------------------------
=== LỌC CỔ PHIẾU NĂM 2022 ===
Tổng số cổ phiếu ban đầu: 1664
Sau lọc marketcap > 1,000,000: 371 cổ phiếu
Sau lọc EPS_change > 0: 192 cổ phiếu
Sau lọc PE < PE trung bình ngành: 95 cổ phiếu
Sau 

In [3]:
# ========= Helper: ranking an toàn, direction = 'desc' nghĩa là giá trị lớn tốt hơn =========
def _rank_pct(s: pd.Series, direction='desc'):
    s = s.replace([np.inf, -np.inf], np.nan)
    if s.isna().all():
        return pd.Series(0.5, index=s.index)
    if direction == 'desc':
        return s.rank(pct=True, ascending=False).fillna(0.5)
    else:
        return s.rank(pct=True, ascending=True).fillna(0.5)

def _pb_proximity_to_1(pb_series: pd.Series):
    # điểm càng cao khi pb ~ 1
    x = pb_series.replace([np.inf, -np.inf], np.nan)
    prox = - (x - 1.0).abs()  # càng gần 0 càng tốt -> giá trị càng lớn càng tốt
    return _rank_pct(prox, direction='desc')

# ========= Cấu hình ngành phòng thủ =========
DEFENSIVE_SECTORS = {
    'Điện, nước & xăng dầu khí đốt',      # Utilities
    'Thực phẩm và đồ uống',               # Consumer Staples
    'Bảo hiểm',                           # Insurance
}

def select_top5_by_style(df_screened: pd.DataFrame, n_growth=3, n_def=2, random_state=42):
    """
    Nhận vào DataFrame đã qua screen_stocks (đã sạch logic),
    trả về 5 mã: 3 Growth + 2 Defensive (có cột 'bucket').
    """
    if df_screened is None or len(df_screened) == 0:
        return pd.DataFrame(columns=['Mã','bucket'])

    df = df_screened.copy()

    # Đảm bảo cột bắt buộc tồn tại
    for col in ['roe','marketcap','pe','pb','eps','avg20_volume']:
        if col not in df.columns:
            df[col] = np.nan

    # ==== Tính điểm Growth ====
    # Ưu tiên EPS_change nếu có, fallback sang eps
    if 'EPS_change' in df.columns and df['EPS_change'].notna().any():
        growth_eps_component = _rank_pct(df['EPS_change'], direction='desc')
    else:
        # 2020 không có EPS_change: dùng eps làm proxy tăng trưởng
        growth_eps_component = _rank_pct(df['eps'], direction='desc')

    growth_roe_component = _rank_pct(df['roe'], direction='desc')
    growth_mc_component  = _rank_pct(df['marketcap'], direction='desc')

    # Trọng số: EPS_change 0.55, ROE 0.35, Marketcap 0.10
    df['growth_score'] = (
        0.55*growth_eps_component +
        0.35*growth_roe_component +
        0.10*growth_mc_component
    )

    # ==== Tính điểm Defensive ====
    # Ngành phòng thủ được cộng bonus
    sector_col = 'Ngành ICB - cấp 2' if 'Ngành ICB - cấp 2' in df.columns else None
    if sector_col:
        df['_sector_def'] = df[sector_col].isin(DEFENSIVE_SECTORS).astype(float)
    else:
        df['_sector_def'] = 0.0

    def_pe_component  = _rank_pct(df['pe'], direction='asc')   # PE càng thấp càng tốt
    def_pb_component  = _pb_proximity_to_1(df['pb'])           # PB ~ 1 càng tốt
    def_mc_component  = _rank_pct(df['marketcap'], direction='desc')  # vốn hóa lớn ổn định hơn

    # Trọng số: Sector 0.35, Marketcap 0.30, PE 0.20, PB~1 0.15
    df['defensive_score'] = (
        0.35*df['_sector_def'] +
        0.30*def_mc_component +
        0.20*def_pe_component +
        0.15*def_pb_component
    )

    # ==== Chọn 3 Growth ====
    growth = df.sort_values(
        by=['growth_score','marketcap'],
        ascending=[False, False]
    ).head(n_growth)

    # ==== Chọn 2 Defensive (không trùng với Growth) ====
    remaining = df[~df['Mã'].isin(growth['Mã'])]
    defensive = remaining.sort_values(
        by=['defensive_score','marketcap'],
        ascending=[False, False]
    ).head(n_def)

    # Nếu thiếu số lượng do dữ liệu ít, bổ sung từ phần còn lại theo tổng điểm an toàn
    if len(growth) < n_growth:
        needed = n_growth - len(growth)
        fill = remaining[~remaining['Mã'].isin(defensive['Mã'])] \
            .sort_values(['growth_score','marketcap'], ascending=[False, False]) \
            .head(needed)
        growth = pd.concat([growth, fill], ignore_index=True)

    if len(defensive) < n_def:
        needed = n_def - len(defensive)
        rest = df[~df['Mã'].isin(pd.concat([growth, defensive])['Mã'])]
        fill = rest.sort_values(['defensive_score','marketcap'], ascending=[False, False]).head(needed)
        defensive = pd.concat([defensive, fill], ignore_index=True)

    growth = growth.copy();     growth['bucket'] = 'Growth'
    defensive = defensive.copy(); defensive['bucket'] = 'Defensive'

    picks = pd.concat([growth, defensive], ignore_index=True)

    # Sắp xếp hiển thị gọn gàng
    display_cols = [c for c in [
        'Mã','Sàn', sector_col, 'marketcap','price',
        'eps', 'EPS_change' if 'EPS_change' in df.columns else None,
        'pe','pb','roe','avg20_volume','sector_pe_avg' if 'sector_pe_avg' in df.columns else None,
        'growth_score','defensive_score','bucket'
    ] if c and c in picks.columns]

    picks = picks[display_cols] \
        .sort_values(['bucket','marketcap'], ascending=[True, False]) \
        .reset_index(drop=True)

    # Dọn cột tạm
    for col in ['_sector_def']:
        if col in picks.columns:
            picks.drop(columns=[col], inplace=True, errors='ignore')
    return picks

def select_top5_for_all_years(screen_results_dict: dict, n_growth=3, n_def=2):
    """
    Nhận vào dict {year: df_screened} từ screen_multiple_years,
    trả về dict {year: df_picks_5mã} có cột 'bucket'.
    """
    out = {}
    for y, df_sc in screen_results_dict.items():
        out[y] = select_top5_by_style(df_sc, n_growth=n_growth, n_def=n_def)
    return out

# ========= Ví dụ dùng ngay sau khi bạn đã có `screening_results` =========
if __name__ == "__main__":
    # ... (giữ nguyên phần bạn đã đọc CSV, chạy screen_multiple_years ở trên)
    top5_each_year = select_top5_for_all_years(screening_results, n_growth=3, n_def=2)

    for year, picks in top5_each_year.items():
        print(f"\n=== 5 MÃ ƯU TIÊN ĐẦU TƯ CHO NĂM {year +1} (3 Growth + 2 Defensive) ===")
        if len(picks) == 0:
            print("Không đủ dữ liệu sau lọc.")
        else:
            print(picks.round(4))
            print("Growth:", picks[picks['bucket']=='Growth']['Mã'].tolist())
            print("Defensive:", picks[picks['bucket']=='Defensive']['Mã'].tolist())



=== 5 MÃ ƯU TIÊN ĐẦU TƯ CHO NĂM 2021 (3 Growth + 2 Defensive) ===
    Mã    Sàn     Ngành ICB - cấp 2  marketcap     price        eps      pe  \
0  VLC  UPCoM  Thực phẩm và đồ uống  2397837.0  36377.53  4667.7712  7.7933   
1  FMC   HOSE  Thực phẩm và đồ uống  1760679.0  33905.55  4351.3883  7.7919   
2  IJC   HOSE          Bất động sản  3358883.0  18806.11  2071.0590  9.0804   
3  TDC   HOSE          Bất động sản  1534999.0  15350.00  2022.0602  7.5913   
4  PRE    HNX              Bảo hiểm  1456000.0  20000.00  2007.5584  9.9624   

       pb     roe  avg20_volume  sector_pe_avg  growth_score  defensive_score  \
0  1.5608  0.1999      112.2789        14.9529        0.3905           0.7214   
1  1.6296  0.2090      521.2000        14.9529        0.4500           0.8048   
2  1.6288  0.1787     2288.2800        10.8557        0.7857           0.4095   
3  1.2548  0.1572     1297.0850        10.8557        0.9048           0.3857   
4  1.5472  0.1551      114.5000        12.4862       